<a href="https://colab.research.google.com/github/UNT-CSCE-Project/sentiment-classification-of-amazon-reviews/blob/dev_avijeet/Copy_of_sentiment_classification_amazon_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
! pip install transformers -q

In [5]:
!pip install vaderSentiment

In [7]:

### Connect with google drive

### import csv from google dir
def connect_and_fetch_file():
  from google.colab import drive
  drive.mount('/content/drive')

  file_id = "1ZBezHHtxu8L6pZVeOtaC0-_n3MjOYnGR"

  file_dir = f'https://drive.google.com/uc?id={file_id}'
  return file_dir
file_dir = connect_and_fetch_file()



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import pandas as pd

df = pd.read_csv(file_dir)


<ipython-input-8-d4830fe90155>:3: DtypeWarning: Columns (1,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_dir)


In [9]:
df_updated = df.copy()  # Create a copy to keep the original DataFrame
df_updated.columns = [col[len('reviews.'):] if col.startswith('reviews.') else col for col in df.columns]


In [10]:
df_updated=df_updated[[ 'doRecommend',  'numHelpful', 'rating',  'text', 'title']]

In [11]:
from textblob import TextBlob

def analyze_sentiment(text):
    analysis = TextBlob(text)
    return analysis.sentiment.polarity

def sentiment_scores(text):
    sentiment = analyze_sentiment(text)
    if sentiment > 0.5:
        return 5
    elif 0.2 < sentiment <= 0.5:
        return 4
    elif -0.2 <= sentiment <= 0.2:
        return 3
    elif -0.5 <= sentiment < -0.2:
        return 2
    else:
        return 1



In [12]:
sentiments = df_updated[df_updated['doRecommend'].isna() & df_updated['rating'].isna()]['text'].apply(sentiment_scores)

In [13]:
df_updated['rating'].fillna(df_updated[df_updated['doRecommend'].isna() & df_updated['rating'].isna()]['text'].apply(sentiment_scores), inplace=True)

In [14]:
df_updated['text' ].fillna('Five Stars', inplace=True)

In [15]:
df_updated['doRecommend'].fillna(False, inplace=True)
df_updated['numHelpful'].fillna(0, inplace=True)

In [16]:
df_updated[df_updated['title'].isna()].iloc[:]

,doRecommend,numHelpful,rating,text,title
3083,False,0.0,5.0,A must have item.,NaN
21953,True,1.0,5.0,I like it. I use mostly for music.,NaN
25515,True,0.0,5.0,I was totally stoked about this product becaus...,NaN
28578,True,0.0,5.0,I'm in LOVE with ALEXA!!,NaN
28680,True,1.0,5.0,"Love it ! I started with a kindle,then a fire ...",NaN


In [17]:
df_updated['numHelpful'] = df_updated['numHelpful'].astype('int')
df_updated['rating'] = df_updated['rating'].astype('int')

In [18]:
df_updated = df_updated.drop('title', axis=1)

In [19]:
df_updated

,doRecommend,numHelpful,rating,text
0,True,0,5,This product so far has not disappointed. My c...
1,True,0,5,great for beginner or experienced person. Boug...
2,True,0,5,Inexpensive tablet for him to use and learn on...
3,True,0,4,I've had my Fire HD 8 two weeks now and I love...
4,True,0,5,I bought this for my grand daughter when she c...
...,...,...,...,...
34655,False,0,3,This is not appreciably faster than any other ...
34656,False,0,1,Amazon should include this charger with the Ki...
34657,False,0,1,Love my Kindle Fire but I am really disappoint...
34658,False,0,1,I was surprised to find it did not come with a...


## Contradictions

In [20]:
contractions_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot",
                   "can't've": "cannot have", "'cause": "because", "could've": "could have",
                   "couldn't": "could not", "couldn't've": "could not have","didn't": "did not",
                   "doesn't": "does not", "don't": "do not", "hadn't": "had not",
                   "hadn't've": "had not have", "hasn't": "has not", "haven't": "have not",
                   "he'd": "he would", "he'd've": "he would have", "he'll": "he will",
                   "he'll've": "he he will have", "he's": "he is", "how'd": "how did",
                   "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                   "I'd": "I would", "I'd've": "I would have", "I'll": "I will",
                   "I'll've": "I will have","I'm": "I am", "I've": "I have",
                   "i'd": "i would", "i'd've": "i would have", "i'll": "i will",
                   "i'll've": "i will have","i'm": "i am", "i've": "i have",
                   "isn't": "is not", "it'd": "it would", "it'd've": "it would have",
                   "it'll": "it will", "it'll've": "it will have","it's": "it is",
                   "let's": "let us", "ma'am": "madam", "mayn't": "may not",
                   "might've": "might have","mightn't": "might not","mightn't've": "might not have",
                   "must've": "must have", "mustn't": "must not", "mustn't've": "must not have",
                   "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                   "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                   "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would",
                   "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have",
                   "she's": "she is", "should've": "should have", "shouldn't": "should not",
                   "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                   "this's": "this is",
                   "that'd": "that would", "that'd've": "that would have","that's": "that is",
                   "there'd": "there would", "there'd've": "there would have","there's": "there is",
                   "they'd": "they would", "they'd've": "they would have", "they'll": "they will",
                   "they'll've": "they will have", "they're": "they are", "they've": "they have",
                   "to've": "to have", "wasn't": "was not", "we'd": "we would",
                   "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have",
                   "we're": "we are", "we've": "we have", "weren't": "were not",
                   "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                   "what's": "what is", "what've": "what have", "when's": "when is",
                   "when've": "when have", "where'd": "where did", "where's": "where is",
                   "where've": "where have", "who'll": "who will", "who'll've": "who will have",
                   "who's": "who is", "who've": "who have", "why's": "why is",
                   "why've": "why have", "will've": "will have", "won't": "will not",
                   "won't've": "will not have", "would've": "would have", "wouldn't": "would not",
                   "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                   "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                   "you'd": "you would", "you'd've": "you would have", "you'll": "you will",
                   "you'll've": "you will have", "you're": "you are", "you've": "you have" }

In [21]:
import re
contractions_re = re.compile('(%s)'%'|'.join(contractions_dict.keys()))
def expand_contractions(s, contractions_dict=contractions_dict):
    if isinstance(s, str):
        def replace(match):
            return contractions_dict[match.group(0)]

        return contractions_re.sub(replace, s)
    else:
        return s

In [22]:
df_updated['expanded_text']  = df_updated['text'].apply(expand_contractions)

In [30]:

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [35]:
import string
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

def process_text(text):
    if isinstance(text, str):
        sentences = sent_tokenize(text)

        stop_words = set(stopwords.words('english'))
        result_text = [
            ' '.join(
                WordNetLemmatizer().lemmatize(word.lower())
                for word in word_tokenize(sentence) if word.lower() not in stop_words and word.isalpha()
            )
            for sentence in sentences
        ]


        result_text = ' '.join(result_text)

        return result_text
    else:
        return ''


In [36]:
df_updated['processed_text'] = df_updated['expanded_text'].apply(process_text)

# references


* https://towardsdatascience.com/text-normalization-for-natural-language-processing-nlp-70a314bfa646


